# Geração do Dataset SintéticoEste notebook é responsável por criar um dataset sintético de imagens de caracteres (visografemas) a partir da fonte ELiS. O processo inclui:- **Configuração de Parâmetros**: Define o tamanho das imagens, a quantidade de amostras por caractere e a divisão entre conjuntos de treino, validação e teste.- **Extração de Caracteres da Fonte**: Identifica todos os caracteres mapeados na fonte `elis.ttf`.- **Renderização e Augmentation**: Para cada caractere, gera uma imagem base e aplica uma série de transformações (rotação, zoom, ruído, etc.) para criar variações, aumentando a robustez do modelo de OCR.- **Estruturação do Dataset**: Organiza as imagens geradas em uma estrutura de diretórios compatível com frameworks de aprendizado de máquina, como TensorFlow e PyTorch (`dataset/{train,validation,test}/{classe}`).

## Importação de BibliotecasCélula dedicada à importação de todas as bibliotecas necessárias para a geração do dataset, incluindo manipulação de arquivos (`os`, `pathlib`), processamento de imagens (`PIL`, `numpy`), data augmentation (`imgaug`) e análise de fontes (`fontTools`).

In [ ]:
import osimport shutilimport randomfrom PIL import Image, ImageFont, ImageDrawimport numpy as npimport imgaug as iaimport imgaug.augmenters as iaafrom fontTools.ttLib import TTFontfrom pathlib import Pathfrom tqdm import tqdm

## Localização da Raiz do ProjetoA função `find_project_root` localiza o diretório raiz do projeto para garantir que os caminhos para fontes e dados sejam resolvidos corretamente, independentemente de onde o notebook é executado.

In [6]:
def find_project_root(markers=(pyproject.toml, .git)) -> Path:    '''    Descobre a raiz do projeto subindo diretórios até encontrar um marcador.    Parâmetros:        markers (tuple): Nomes de arquivos/pastas que indicam a raiz (ex.: pyproject.toml, .git).    Retorna:        pathlib.Path: Aponta para a raiz do projeto, ou o diretório atual como fallback.    '''    p = Path.cwd().resolve()    for parent in [p, *p.parents]:        if any((parent / m).exists() for m in markers):            return parent    return pROOT = find_project_root()

## Configurações GeraisEsta célula centraliza todos os parâmetros de configuração para a geração do dataset, como caminhos de diretórios, dimensões das imagens, tamanho da fonte e proporções para a divisão dos dados.

In [10]:
# ConfiguraçõesDATA_DIR = ROOT / dataFONT_PATH = DATA_DIR / external / elis.ttfOUTPUT_DIR = DATA_DIR / raw / datasetIMAGE_SIZE = (64, 64)FONT_SIZE = 48SAMPLES_PER_SYMBOL = 1000# Divisão do datasetTRAIN_RATIO = 0.7VALIDATION_RATIO = 0.15#TEST_RATIO será o restante

## Análise da Fonte e Extração de CaracteresCarrega a fonte `elis.ttf`, conta o número total de glifos e extrai os codepoints Unicode que serão usados como classes no dataset. Cada codepoint é convertido para o formato `U+XXXX`, que nomeará as pastas de cada classe.

In [8]:
font = TTFont(FONT_PATH)# número de glifos (glyphs) definidos na fonte# Glifos são as formas visuais — os desenhos reais dentro da fonte.# Ex.: um “a” romano, uma variante cursiva, uma ligadura “ffi”, ou um acento separado são todos glifos.# Contagem típica: inclui glifos de base, alternates, ligaduras, marcas, .notdef, etc.glyph_count = len(font.getGlyphOrder())print(f'Número de glifos na fonte: {glyph_count}')

Número de glifos na fonte: 149

In [9]:
# Cria a lista de codepoints (códigos Unicode) em hexadecimal para os glifoscodepoints = []for table in font['cmap'].tables:    if table.isUnicode():        codepoints.extend(table.cmap.keys())codepoints = list(set(codepoints))  # Remove duplicatasprint(f'Número de codepoints únicos: {len(codepoints)}')CLASS_NAMES = [f'U+{cp:04X}' for cp in codepoints]# Tenta fechar a fontetry:    font.close()except Exception:    pass

Número de codepoints únicos: 146

## Definição de AugmentationsConfigura uma sequência de transformações de imagem (`augmentation`) com a biblioteca `imgaug`. Essas transformações, como zoom, rotação, desfoque e ruído, são aplicadas aleatoriamente às imagens base para criar um dataset mais variado e robusto.

In [11]:
# --- Definição das Augmentations com imgaug ---# Define uma sequência de transformações a serem aplicadasaugmentation_seq = iaa.Sequential(    [        iaa.Affine(            scale={x: (0.8, 1.2), y: (0.8, 1.2)},  # Zoom de 80% a 120%            translate_percent={x: (-0.1, 0.1), y: (-0.1, 0.1)},  # Translação            rotate=(-15, 15),  # Rotação de -15 a 15 graus            shear=(-8, 8),  # Cisalhamento            cval=255,  # Cor de fundo branco            mode='constant',        ),        # iaa.Multiply((0.7, 1.3), per_channel=0.2),         # Altera o brilho        iaa.Multiply((0.85, 1.15), per_channel=0.2),  # Reduz variação para não escurecer muito        iaa.GaussianBlur(sigma=(0, 0.8)),  # Desfoque Gaussiano        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.03 * 255), per_channel=0.5),  # Ruído    ],    random_order=True,)  # Aplica as transformações em ordem aleatória

## Funções de Geração de ImagemDuas funções auxiliares são definidas:- `render_glyph_bitmap`: Renderiza um glifo em uma imagem PIL, centralizando-o. (Esta é uma versão melhorada da função do notebook anterior).- `create_base_image`: Utiliza a função anterior para gerar a imagem de um caractere, redimensiona para o tamanho final do dataset e a converte para um array NumPy.

In [ ]:
from PIL import Imagefrom typing import Uniondef render_glyph_bitmap(    font_path: Union[str, os.PathLike],    char_or_codepoint: Union[int, str],    image_size: int = 256,    font_px: int | None = None,    bgcolor: int = 255,    fgcolor: int = 0,) -> Image.Image:    '''    Renderiza um glifo de uma fonte TTF/OTF em uma imagem quadrada em escala de cinza (modo 'L').    Parâmetros:        font_path: caminho para o arquivo de fonte (TTF/OTF). Aceita str ou PathLike.        char_or_codepoint: inteiro Unicode (ex.: 0x0041) ou caractere único (ex.: A).        image_size: lado do quadrado de saída, em pixels (ex.: 256).        font_px: tamanho do corpo da fonte em pixels; se None, usa 80% de image_size.        bgcolor: cor de fundo (0..255), padrão 255 (branco).        fgcolor: cor do glifo (0..255), padrão 0 (preto).    Retorna:        PIL.Image.Image: No modo 'L' com o glifo centralizado no canvas.    Erros:        FileNotFoundError: se o arquivo de fonte não existir.        ValueError: se for passada string com mais de um caractere.        OSError: se a fonte não puder ser carregada pelo Pillow.    Observações:        - Centraliza o glifo usando getbbox para calcular deslocamento exato.        - Se a fonte não tiver o glifo específico, a renderização pode recair em fallback da fonte.    '''    # Normaliza o caminho e valida existência    font_path = str(font_path)    if not os.path.exists(font_path):        raise FileNotFoundError(font_path)    # Normaliza o caractere    if isinstance(char_or_codepoint, int):        ch = chr(char_or_codepoint)    else:        ch = str(char_or_codepoint)        if len(ch) != 1:            raise ValueError(Quando str, 'char_or_codepoint' deve ter exatamente 1 caractere.)    # Define tamanho da fonte se não informado    if font_px is None:        font_px = int(image_size * 0.8)    # Carrega a fonte e renderiza centralizado    font = ImageFont.truetype(font_path, size=font_px)    img = Image.new('L', (image_size, image_size), color=bgcolor)    draw = ImageDraw.Draw(img)    bbox = font.getbbox(ch)  # (x0, y0, x1, y1) relativo à origem    w = bbox[2] - bbox[0]    h = bbox[3] - bbox[1]    x = (image_size - w) // 2 - bbox[0]    y = (image_size - h) // 2 - bbox[1]    draw.text((x, y), ch, font=font, fill=fgcolor)    # FreeTypeFont não exige close; mantendo try/except por compatibilidade    try:        font.close()    except Exception:        pass    return img

In [ ]:
def create_base_image(symbol: int) -> np.ndarray:    '''    Renderiza um símbolo e o redimensiona para o tamanho padrão do dataset.    Esta função utiliza `render_glyph_bitmap` para criar uma imagem de alta resolução    de um glifo e, em seguida, a redimensiona para o `IMAGE_SIZE` configurado,    usando um filtro de alta qualidade para preservar detalhes.    Args:        symbol (int): O codepoint Unicode do caractere a ser renderizado.    Returns:        np.ndarray: A imagem do glifo como um array NumPy em escala de cinza.    '''    # Usa a configuração global FONT_PATH (normalizada para str)    image = render_glyph_bitmap(str(FONT_PATH), symbol, image_size=256)    # Redimensiona com o melhor filtro disponível    try:        resample_filter = Image.Resampling.LANCZOS  # Pillow >= 9.1    except AttributeError:        resample_filter = Image.LANCZOS             # Pillow < 9.1    image = image.resize(IMAGE_SIZE, resample=resample_filter)    return np.array(image, copy=False)

In [32]:
# Exemplo de uso# create_base_image(0x0030)

## Preparação dos Diretórios de SaídaA função `setup_directories` prepara a estrutura de pastas para o dataset. Ela limpa o diretório de saída principal para evitar dados antigos e cria as subpastas `train`, `validation` e `test`, cada uma contendo diretórios para todas as classes.

In [ ]:
def setup_directories() -> None:    '''    Limpa e recria a estrutura de diretórios para o dataset.    A estrutura final será `OUTPUT_DIR/{train,validation,test}/U+XXXX`, onde `U+XXXX`    corresponde a cada classe (caractere) a ser gerada.    Atenção: Esta função deleta o diretório `OUTPUT_DIR` se ele já existir.    '''    if OUTPUT_DIR.exists():        shutil.rmtree(OUTPUT_DIR)    for split in ['train', 'validation', 'test']:        for class_name in CLASS_NAMES:            (OUTPUT_DIR / split / class_name).mkdir(parents=True, exist_ok=True)    print(Estrutura de diretórios criada com sucesso.)

## Geração e Salvamento do DatasetEste é o loop principal do notebook. Ele itera sobre cada caractere (codepoint), gera a imagem base, aplica as augmentations definidas para criar o número desejado de amostras e salva cada imagem no diretório correspondente (treino, validação ou teste) de forma aleatória, respeitando as proporções definidas.

In [ ]:
if not os.path.exists(FONT_PATH):    raise FileNotFoundError(f'Fonte não encontrada: {FONT_PATH}')    #returnsetup_directories()for i, symbol in enumerate(tqdm(codepoints, desc=Gerando dataset)):    class_name = f'U+{symbol:04X}'    base_image = create_base_image(symbol)    for j in range(SAMPLES_PER_SYMBOL):        augmented_image = augmentation_seq(image=base_image)        img_pil = Image.fromarray(augmented_image)        # Decide a divisão do dataset        rand_val = random.random()        if rand_val < TRAIN_RATIO:            split = 'train'        elif rand_val < TRAIN_RATIO + VALIDATION_RATIO:            split = 'validation'        else:            split = 'test'        img_filename = os.path.join(OUTPUT_DIR, split, class_name, f'{class_name}_{j:04d}.png')        img_pil.save(img_filename)print(Dataset gerado com sucesso.)